## Welcome to the Planetary Data Reader Example Jupyter Notebook!

The Planetary Data Reader (`pdr`) is a Python package that provides a single, straightforward interface to planetary science observational data. It is currently under active development and will eventually support almost all data hosted by the Planetary Data System (PDS). The basic command is just `pdr.read(fn)`, where `fn` is either an observational data file or its detached label file (if applicable).

This notebook demonstrates basic usage and key features of `pdr`.

_**Note:** This notebook must download data from the PDS and therefore requires an internet connection. `pdr` does not require an internet connection to be used locally when the data are already present._

In [ ]:
# imports
# glob, os, and requests are used to check paths and retrieve remote 
# data in this notebook. you do not need to import them for most uses of pdr.
import glob
import os 
import requests 

# importing the pdr module itself is mandatory for most uses of pdr.
import pdr  

First, we'll look at an image from LROC (which uses the older PDS3 standards and has an attached label).

In [ ]:
# This is the URL for the data file on the PDS server
img_url = 'https://pds.lroc.asu.edu/data/LRO-L-LROC-5-RDR-V1.0/LROLRC_2001/DATA/BDR/NAC_ROI/NECTARISLOA/NAC_ROI_NECTARISLOA_E176S0413_20M.IMG'
filename = img_url.split('/')[-1]
# This retrieves the file and saves it to the current working directory.
# It will print the total data volume in bytes (8227536) when it completes.
if not os.path.exists(filename):
    req = requests.get(img_url)
    open(filename, 'wb').write(req.content)
    print(len(req.content))

### Reading image data:
Now that we've downloaded our file by running the previous cell, we can easily read the data in with `pdr.read()`. This returns an object with attributes that correspond to each of the data objects defined in the label. Much like a Python `dict`, the attributes of this object can be queried with `__getitem__` notation (["thing"]) and accessed with a `keys()` method, 

Let's read the file and see what kinds of data it contains.

In [ ]:
lroc_data = pdr.read(filename)  # That's the magic function call!
print(f'The keys are {lroc_data.keys()}')

If you are not familiar with PDS data formats, you might be surprised to learn that this "image" file contains three different data objects. Most PDS products have a `LABEL` object, which contains metadata associated with an observation (like observation time, calibration constants, provenance information, etc.). `pdr` interprets `LABEL`s as Parameter Value Language (`PVL`) objects. The `IMAGE` object is an array of observational data values; `pdr` interprets it as a `numpy.ndarray`. `DATA_SET_MAP_PROJECTION`, which is included in some map-projected PDS3 data products, contains information about the map projection.

Let's print the contents of each of these objects, just to see what they contain.

In [ ]:
for key in lroc_data.keys():
    print(f'{key}:')
    print(lroc_data[key])

**Oopsie!** `pdr` threw an `AttributeError` when we attempted to access `DATA_SET_MAP_PROJECTION`. This is intentional. Many files in the PDS have small format (.FMT), catalog (.CAT), or other supplementary files referenced in their labels. These files are usually not stored in the same directory on the PDS's servers as the data and label files. Most of the time, these supplementary files will not be necessary to read the objects you care about: note that we accessed the `LABEL` and `IMAGE` objects with no issues. However, if they are necessary and are not in your filesystem, `pdr` cannot load them and will throw an error when you attempt to access associated data objects.

#### `.show()` convenience method (for visualizing image data):
`pdr` has a convenience method called `.show()` which helps to quickly visualize image data. Null values---typically defined in the label or drawn from a list of universal null values---are masked in cyan, but that doesn't change their value in the data object. This method is solely for visualizing data for browsing or triage purposes. (If you want an array containing the specific values used to render these images, use `data.get_scaled("NAME_OF_OBJECT")`.)

In [ ]:
lroc_data.show()

### Reading table data:
Now let's look at a table-like data file.  The same `pdr.read()` command works, and `pdr` figures out the format of the data and correctly interprets it. Table-like data objects are represented as `pandas.DataFrame`s.

First we'll read a table from the Apollo 15 Heat Flow Experiment that is in the PDS4 standard format. Then we'll read some MRO SHARAD data that is in the PDS3 standard format. We also demonstrate that data can be opened with either a detached label file or a data file, although **label files are preferred.**

_**Note:** Currently, pdr does not use the same internal functions to open PDS4 and PDS3 data. PDS4 data is less optimized---it reads data accurately, but more slowly---and **we recommend using PDS3 labels whenever both are available** due to this._

In [ ]:
apollo_lbl

In [ ]:
apollo_fn

In [ ]:
apollo_lbl

In [ ]:
# These are URLs for Apollo 15 HFE data and label files
apollo_url = 'https://pds-geosciences.wustl.edu/lunar/'\
             'urn-nasa-pds-a15_17_hfe_concatenated/data/split/a15p1f4_split.tab'
apollo_lbl = 'https://pds-geosciences.wustl.edu/lunar/'\
             'urn-nasa-pds-a15_17_hfe_concatenated/data/split/a15p1f4_split.xml'
# This downloads the data, same way as above
apollo_fn = apollo_url.split('/')[-1]
if not os.path.exists(apollo_fn):
    req = requests.get(apollo_url)
    open(apollo_fn, 'wb').write(req.content)
apollo_lbl_fn = apollo_lbl.split('/')[-1]
if not os.path.exists(apollo_lbl):
    reqlbl = requests.get(apollo_lbl)
    open(apollo_lbl_fn, 'wb').write(reqlbl.content)
apollo_from_data_file = pdr.read(apollo_fn)
apollo_from_lbl_file = pdr.read(apollo_lbl_fn)
# This checks that the outputs of `pdr` are identical 
# whether you pass it the data file or the label file.
print('Do the data file and label file produce identical outputs?',
      all(apollo_from_data_file['a15p1f4_split']==apollo_from_lbl_file['a15p1f4_split']))

Tables are interpreted as pandas dataframes when read from PDS3 products and as PDS_ndarrays (subclassed numpy arrays) when read from PDS4 labels (as shown here). PDS4 labels are output as dictionaries. 

In [ ]:
print(f'The keys are {apollo_from_data_file.keys()}')
for key in apollo_from_data_file.keys():
    print(f'{key}:')
    print(type(apollo_from_data_file[key]))
    print(apollo_from_data_file[key])

An example with a MRO-RSS file (a PDS3 formatted table) can be seen below. Note again the difference in table types returned from using PDS3 and PDS4 labels: pandas dataframes vs. PDS_ndarray.

Here we also demonstrate the use of the `lazy=` keyword. When passed `lazy=True` and a detached label file, `pdr` only reads the label file, allowing you to load each data object at your leisure.

In [ ]:
mrorss_url = 'https://pds-geosciences.wustl.edu/mro/mro-m-rss-5-sdp-v1/'\
             'mrors_1xxx/data/shadr/jgmro_110b2_sha.tab'
mrorss_lbl_url = 'https://pds-geosciences.wustl.edu/mro/mro-m-rss-5-sdp-v1/'\
                 'mrors_1xxx/data/shadr/jgmro_110b2_sha.lbl'
mrorss_fn = mrorss_url.split('/')[-1]
if not os.path.exists(mrorss_fn):
    req = requests.get(mrorss_url)
    open(mrorss_fn, 'wb').write(req.content)
mrorss_lbl = mrorss_lbl_url.split('/')[-1]
if not os.path.exists(mrorss_lbl):
    req_lbl = requests.get(mrorss_lbl_url)
    open(mrorss_lbl, 'wb').write(req_lbl.content)

mrorss_data=pdr.read(mrorss_lbl, lazy=True)

mrorss_data

In [ ]:
# now we can load individual objects. note that this TABLE 
# no longer appears in the 'not yet loaded' list.
mrorss_data.load('SHADR_COEFFICIENTS_TABLE')
mrorss_data

#### `.dump_browse()` convience method (for outputting browse products of any data type)

Much like the `.show()` method, the `dump_browse()` method can output a masked browse image. However, there are some key differences.

(1) `.dump_browse()` will create a browse file on your computer drive, not a visual output on your display.

(2) While `.show()` only works on array data (meant for images), the `.dump_browse()` feature will output a file for each key.

Let's give it a go:

In [ ]:
mrorss_data.dump_browse()
print(glob.glob('jgmro_110b2_sha*'))

There should now be two new files in the folder you have this jupyter notebook in (visible in the printed `glob` statement above) as well

    -jgmro_110b2_sha_LABEL.lbl
    -jgmro_110b2_sha_SHADR_COEFFICIENTS_TABLE.csv
    
Dumped browse filenames are created from the original filename, plus \_key, where key is the name of the corresponding dictionary key.

Notice that only the keys that were loaded in have corresponding files when running `dump_browse()`.

The label is output as a PVL parsed .lbl file. When PDS labels don't conform to the strict standards of PVL (which they often don't) the \_LABEL.lbl will be accompanied by a \_LABEL.badpvl.txt which is the backup output. It is simply a text file of the LABEL key. In this case the \_LABEL.lbl will be an empty text file.

We can use this same convenience method with the image data from above (requires running the first 2 cells after import in case you jumped here). This will output the image we displayed above as a .jpg and also output a parsed .lbl which will both be saved in the folder the jupyter notebook is executing from.

In [ ]:
lroc_data.dump_browse()